In [78]:
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool
import datetime
import urllib.request
import scrapers
import requests

In [2]:
import itertools
import tqdm.notebook as tqdm


def p_imap_unordered_list(pool, func, iterable):
    iterable = list(iterable)
    temp = tqdm.tqdm(pool.imap_unordered(func, iterable), total=len(iterable))
    return list(temp)


the_time = lambda: datetime.datetime.now().time().strftime("%H:%M:%S")

In [3]:
data = pd.read_csv("scraped.csv").query("team != 'team/'")
riders = set(data["rider"])
teams = set(data["team"])

In [4]:
print(the_time())
with Pool(50) as p:
    pretty_teams = p_imap_unordered_list(p, scrapers.get_pretty_team, teams)
print(the_time())
with Pool(50) as p:
    pretty_riders = p_imap_unordered_list(p, scrapers.get_pretty_rider, riders)
print(the_time())

18:47:03


  0%|          | 0/23429 [00:00<?, ?it/s]

18:58:39


  0%|          | 0/72879 [00:00<?, ?it/s]

19:39:00


In [111]:
out = pd.DataFrame(pretty_riders+pretty_teams)[["name", "url", "year"]]
out["name"] = out["name"].str.strip()
out = out.sort_values(["name", "year"])
out = out[out["name"].astype(bool)] # empties are errors on firstcycling's part

In [92]:
scraped = pd.read_csv("scraped.csv")
with Pool(50) as p:
    teams_to_fix = p_imap_unordered_list(
        p,
        scrapers.team_year_from_rider,
        (
            scraped[scraped["team"].isin(-out.query("year == 1")["url"])]
            .drop_duplicates("team")
            .itertuples(index=False, name=None)
        ),
    )
big_teams_to_fix = {-team:year for d in teams_to_fix for team,year in d.items()}

  0%|          | 0/6880 [00:00<?, ?it/s]

In [117]:
to_replace = out["url"].map(big_teams_to_fix).dropna().convert_dtypes()
out.loc[to_replace.index, "year"] = to_replace
out.to_csv("names.csv", index=False)